In [4]:
import asyncio
from tardis_client import TardisClient, Channel
from datetime import datetime
from datetime import timedelta
import os

In [5]:
# messages as provided by Deribit real-time stream
# sample ticker message
# {
#   "jsonrpc": "2.0",
#   "method": "subscription",
#   "params": {
#     "channel": "ticker.ETH-27DEC19-400-P.raw",
#     "data": {
#       "underlying_price": 181.88,
#       "underlying_index": "ETH-27DEC19",
#       "timestamp": 1569887997259,
#       "stats": {
#         "volume": null,
#         "low": null,
#         "high": null
#       },
#       "state": "open",
#       "settlement_price": 1.41,
#       "open_interest": 16,
#       "min_price": 1.145,
#       "max_price": 1.307,
#       "mark_price": 1.223004,
#       "mark_iv": 109.02,
#       "last_price": 1.195,
#       "interest_rate": 0,
#       "instrument_name": "ETH-27DEC19-400-P",
#       "index_price": 181.19,
#       "greeks": {
#         "vega": 0.17042,
#         "theta": -0.10637,
#         "rho": -0.91829,
#         "gamma": 0.00198,
#         "delta": -0.88711
#       },
#       "bid_iv": 0,
#       "best_bid_price": 0.001,
#       "best_bid_amount": 8,
#       "best_ask_price": 0,
#       "best_ask_amount": 0,
#       "ask_iv": 0
#     }
#   }
# }

In [10]:
tardis_client = TardisClient(api_key="TD.1p99y7G1So1J2QHl.pluqusoyNBCsXKc.kAgEnv7tJaay1zV.1UUkeXx60EhChSV.d8jRdEBJlpsdT8n.c7Ad")

async def pull_daily_quotes(start_date, end_date):

    async def replay():
        messages = tardis_client.replay(
            exchange="deribit",
            from_date=start_date,
            to_date=end_date,
            filters=[Channel(name="ticker", symbols=[])],
        )

        filename = start_date[0:10].replace('-', '') + ".csv"
        path = os.path.join(os.getcwd(), "_input", "tickers", filename)

        i = 0

        with open(path, "w") as f:
            async for local_timestamp, message in messages:
                instrument_name = message["params"]["data"]["instrument_name"]
                is_option = instrument_name.endswith("-P") or instrument_name.endswith("-C")
                if is_option:
                    data = message["params"]["data"]
                    greeks = data['greeks']

                    for greek, value in greeks.items():
                      data[greek] = value

                    del data['greeks']
                    del data['stats']

                    if i == 0:
                        f.write(','.join(data.keys()) + '\n')
                        i += 1
                    
                    f.write(','.join(map(str, data.values())) + '\n')

            f.close()

    await replay()


async def pull_quotes():

    date_format = "%Y-%m-%d %H:%M"
    initial_start_date = datetime.strptime("2020-07-01 10:59", date_format)
    initial_end_date = initial_start_date + timedelta(minutes=1)

    for days in range(0, 31):
        start_date = datetime.strftime(initial_start_date + timedelta(days=days), date_format)
        end_date = datetime.strftime(initial_end_date + timedelta(days=days), date_format)
        print(f"Pulling the tickers for {start_date}")
        await pull_daily_quotes(start_date, end_date)


await pull_quotes()

Pulling the tickers for 2020-07-05 10:59


HTTPError: HTTP Error 400: {
  "code": 210,
  "message": "Provided API key does not allow access to the data feed for '2020-07-05T10:59:00.000Z' date ('from'+'offset' params combination).",
  "apiKeyAccessInfo": [
    {
      "exchange": "deribit",
      "accessType": "one-off-purchase",
      "from": "2019-04-01T00:00:00.000Z",
      "to": "2020-07-01T00:00:00.000Z",
      "symbols": []
    }
  ]
}